# Analyzing NBA Salaries in relation to performance: finding the most overpaid and underpaid 

## Amolak Singh

The goal of this mini-project is to determine which NBA players will be overpaid and underpaid in the 2018-19 season relative to their performance the prior season. In the past, the NBA may have payed players based more on reputation and less informative statistics but nowadays the use of advanced analytics is more common. The methodology of this project is as follows:
* 1st notebook: 1_nba_data_prep_clustering.ipynb
  1. We will use some advanced analytics pulled from *basketball-reference.com* to cluster players into several different groups based on their performance in the 2017-18 season. This will allow us to group similar players together so we can do more apple to apple comparisions when analyzing their salaries.
  2. After creating the group labels, we will pull salary information from *basketball-reference.com* and clean that data to join with the data above.
* 2nd notebook: 2_nba_results_analysis_clean.ipynb
  3. We wil analyze the salaries in relation to performance. Within each cluster, we will calculate the percentile/rank of each player salary. Players in the bottom quartile will be considered underpaid relative to their performance and players in the upper quartile will be considered overpaid relative to their performance.
  4. We will finish with a simple regression analysis and some final conclusions.

### Imports

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.preprocessing import scale,StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings('ignore')

### Variables to be used later

In [2]:
year = 2018
url = 'https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'.format(year)
salary_url = 'https://www.basketball-reference.com/contracts/players.html'
seed = 6
clusters = 10

In [3]:
not_yet_num_cols = ['Age','G','MP','PER','TS%','3PAr','FTr','ORB%','DRB%','TRB%',
                    'AST%','STL%','BLK%','TOV%','USG%','OWS','DWS','WS','WS/48',
                    'OBPM','DBPM','BPM','VORP']
df_interesting_variables = ['TS%','3PAr','FTr','ORB%','DRB%','TRB%','AST%','STL%',
                            'BLK%','TOV%','USG%','OWS','DWS','WS','WS/48','OBPM',
                            'DBPM','BPM', 'VORP']

### Pulling the player analytics with BeautifulSoup

In [4]:
html = urlopen(url)
soup = BeautifulSoup(html,"lxml")
cols = [th.getText() for th in soup.findAll('tr',limit=3)[0].findAll('th')]
cols.remove('Rk')

In [5]:
rows = soup.findAll('tr')[1:]
dat = [[td.getText() for td in rows[i].findAll('td')] for i in (range(len(rows)))]
df = pd.DataFrame(dat,columns=cols)

In [6]:
df = pd.concat([df[['Player','Pos','Tm',]], df[not_yet_num_cols].apply(pd.to_numeric, errors='ignore')], axis=1)
# removed players who played less than 50 games,  played less than 800 minutes, and had a negative win 
# share impact (per game)
df = df[(df.G >= 50) & (df.MP >= 800) & (df.WS/48 > 0)]
df = df.reset_index()
df.to_csv('full_nba_data')
df.head(5)

,index,Player,Pos,Tm,Age,G,MP,PER,TS%,3PAr,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,0,Alex Abrines,SG,OKC,24.0,75.0,1134.0,9.0,0.567,0.759,...,7.4,12.7,1.3,1.0,2.2,0.094,-0.5,-1.7,-2.2,-0.1
1,1,Quincy Acy,PF,BRK,27.0,70.0,1359.0,8.2,0.525,0.800,...,13.3,14.4,-0.1,1.1,1.0,0.036,-2.0,-0.2,-2.2,-0.1
2,2,Steven Adams,C,OKC,24.0,76.0,2487.0,20.6,0.630,0.003,...,13.2,16.7,6.7,3.0,9.7,0.187,2.2,1.1,3.3,3.3
3,3,Bam Adebayo,C,MIA,20.0,69.0,1368.0,15.7,0.570,0.021,...,13.6,15.9,2.3,1.9,4.2,0.148,-1.6,1.8,0.2,0.8
4,6,LaMarcus Aldridge,C,SAS,32.0,75.0,2509.0,25.0,0.570,0.068,...,6.8,29.1,7.4,3.5,10.9,0.209,3.0,0.3,3.3,3.3


In [7]:
df_k_means = df[df_interesting_variables]
df_k_means.to_csv('clustering_nba_data')

### Clustering our players and creating group labels

Chose 10 clusters because too few clusters (5-7) did not lead to good enough differentiation while too many clusters (13-18) lead to too much differentiation. 10 splits the difference well. 

In [8]:
scaler = StandardScaler()
kmeans = KMeans(n_clusters=clusters, random_state = seed)
pipeline = make_pipeline(scaler, kmeans)
pipeline.fit_transform(df_k_means)

array([[  3.56332261,   7.84325425,   3.67772102, ...,  10.45145737,
         10.70714706,   5.02250867],
       [  2.49836828,   7.24564669,   3.84061743, ...,  11.09887219,
         10.70085919,   5.70821354],
       [  9.26047796,   4.11616408,   6.47334072, ...,   7.06909479,
          4.23229336,   7.00997202],
       ..., 
       [  3.46497425,   8.64435282,   4.4362572 , ...,  11.11002985,
         11.48718358,   5.25931461],
       [  5.38018047,   5.29422345,   3.57714362, ...,   8.09604362,
          7.15397946,   4.77949839],
       [  4.87547455,   4.00625033,   3.70094424, ...,   9.91357334,
          8.05163245,   5.5730642 ]])

In [9]:
labs = pipeline.predict(df_k_means)
results_df = pd.concat([df[['Player','Pos','Tm',]], df_k_means], axis=1)
results_df['label'] = labs.tolist()

### Pulling and cleaning salary data

In [10]:
salaries = pd.read_html(salary_url, header=1)[0]
salaries = salaries[salaries['Rk'] != 'Rk']
salaries = salaries[['Player', '2018-19', 'Guaranteed']]

In [11]:
# removing commmas and dollar signs from the money columns
for col in ['2018-19','Guaranteed']:
    salaries[col] = salaries[col].replace('[\$,]', '', regex=True).astype(float)

In [12]:
salaries.to_csv('player_salaries')
salaries.head()

In [13]:
combined_df = pd.merge(results_df, salaries, how='left', left_on=['Player'], right_on=['Player'])

In [14]:
# we will the null salaries with 0. These players are either free agents who have not signed new deals yet, players 
# who wil be playing abroad, or potentially retirees
combined_df = combined_df.fillna(0)
combined_df.head()
combined_df.to_csv('joined_nba_data')